In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data.  "." means current directory
# untar true will unzip it

In [ ]:
data_dir

In [ ]:
import pathlib  

data_dir = pathlib.Path(data_dir)  # Convert 'data_dir' to a Path object for easier path handling
data_dir  

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
roses[:5]

In [ ]:
PIL.Image.open(str(roses[5]))

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[4]))

In [ ]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [ ]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

### Using opencv to read image

In [ ]:
X, y = [], []  # Initialize empty lists for images (X) and their labels (y)

# Loop through each flower category and its image paths
for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))  # Read the image using OpenCV
        resized_img = cv2.resize(img, (180, 180))  # Resize image to 180x180 pixels
        X.append(resized_img)  # Add the processed image to the list
        y.append(flowers_labels_dict[flower_name])  # Add the corresponding label


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Scaling Images 

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

## CNN

In [ ]:
num_classes = 5

model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train, y_train, epochs=30)  

In [ ]:
model.evaluate(X_test,y_test)

Training accuracy is .99 and testing is .63 so, clearly is an overfit 

### Trying to check prediction

In [ ]:
predictions = model.predict(X_test)
predictions

In [ ]:
score = tf.nn.softmax(predictions[0])
print(np.argmax(score))

In [ ]:
print(y_test[0])

Seems like it did predicted correctly

## Performing Data Augmentation now

Was crashing from here before

In [ ]:
da = keras.Sequential([layers.RandomZoom(0.9)])

In [ ]:
data_augmentation = keras.Sequential([
    # Randomly flip images horizontally during training
    layers.RandomFlip(mode="horizontal", seed=None, input_shape=(180, 180, 3)),

    # Randomly rotate images by up to 10%
    layers.RandomRotation(0.1),

    # Randomly zoom in on images by up to 10%
    layers.RandomZoom(0.1),
])
